# Лабораторная работа №2.Поиск ассоциативных правил(Last FM hometask)
https://www.kaggle.com/ravichaubey1506/lastfm <br>
## Зямзина Анастасия и Лукашева Екатерина
 <b>3530903_70302</b>
 
Задание: 
1. Обработать данные по странам своей группы (совместно): Spain, Portugal, France, Italy, Belgium<br> 
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

## Подготовка данных

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [4]:
countries = ['Spain', 'Portugal', 'France', 'Italy', 'Belgium']
data_countries = pd.DataFrame()
for i in range(len(countries)):
    df2 = data[data.country == countries[i]]
    data_countries= pd.concat([data_countries, df2], ignore_index=True)
data_countries.country.value_counts()

Spain       9322
France      5962
Italy       5717
Belgium     3331
Portugal    2882
Name: country, dtype: int64

## Узнаем, какие артисты нравятся каждому пользователю

In [5]:
data_countries.describe()
data_by_user = data_countries.groupby(['user','country'])['artist'].apply(lambda x: ",".join(x)).reset_index()
data_by_user

,user,country,artist
0,6,Portugal,"lily allen,kanye west,sigur rós,pink floyd,ste..."
1,12,Italy,"queen,the beatles,lynyrd skynyrd,neil young,th..."
2,36,Italy,"m.i.a.,fugazi,johnny cash,clint mansell,elton ..."
3,47,Italy,"pj harvey,radiohead,zero 7,einstürzende neubau..."
4,74,Spain,"in extremo,nine inch nails,billy talent,panic!..."
...,...,...,...
1414,19690,Italy,"pink floyd,the beatles,the white stripes,depec..."
1415,19693,Belgium,"air,simon & garfunkel,pixies,serge gainsbourg,..."
1416,19697,Belgium,"underworld,pendulum,the prodigy,sufjan stevens..."
1417,19704,Italy,"keane,pinback,coldplay,travis,calexico,doves,d..."


In [6]:
data_by_user['country'].value_counts()

Spain       506
France      327
Italy       304
Belgium     160
Portugal    122
Name: country, dtype: int64

## Подготовка данных для обработки алгоримами (dummy encoding)

In [11]:
for_dummy_enc = data_countries.drop(columns=['country','sex'])
data_to_process = for_dummy_enc.pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)

for_dummy_enc_with_sex = data_countries.drop(columns=['artist','country'])
data_to_process_with_sex = pd.concat([data_to_process, 
                                      for_dummy_enc_with_sex.pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)], 
                                     axis = 1)

## Обработка данных различными алгоритмами

#### Apriori

In [12]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets_apriori = apriori(data_to_process, min_support=0.01, use_colnames=True)

frequent_itemsets_apriori.head()

,support,itemsets
0,0.011276,(2pac)
1,0.020437,(3 doors down)
2,0.029598,(30 seconds to mars)
3,0.012685,(50 cent)
4,0.019732,(65daysofstatic)


#### FP-growth

In [13]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets_fpgrowth = fpgrowth(data_to_process, min_support=0.08, use_colnames=True)

frequent_itemsets_fpgrowth.head()

,support,itemsets
0,0.174066,(muse)
1,0.130374,(pink floyd)
2,0.123326,(metallica)
3,0.096547,(depeche mode)
4,0.088090,(u2)


#### FPMax

In [14]:
from mlxtend.frequent_patterns import fpmax

frequent_itemsets_fpmax = fpmax(data_to_process, min_support=0.05, use_colnames=True)

frequent_itemsets_fpmax

,support,itemsets
0,0.050035,(gorillaz)
1,0.050035,(pj harvey)
2,0.050035,(editors)
3,0.050740,(kaiser chiefs)
4,0.050740,(snow patrol)
...,...,...
67,0.054968,"(pink floyd, radiohead)"
68,0.070472,"(the beatles, radiohead)"
69,0.068358,"(coldplay, muse)"
70,0.080338,"(muse, radiohead)"


## Правила 

Дополнтельные функции

In [15]:
def print_res(algorithm, answer):
    print('Algorithm: ' + algorithm)
    for i in range(len(answer['antecedents'])):
        print(set(answer.iloc[i,0]) , ' -> ', set(answer.iloc[i,1]))

#### 1 правило: 

In [16]:
def first_rule(data_for_this_rule, min_t):
    return association_rules(data_for_this_rule, metric="support", min_threshold=min_t) 

data_first_rule_apriori = first_rule(frequent_itemsets_apriori,0.005)
print(data_first_rule_apriori.head())
data_first_rule_fpgrowth = first_rule(frequent_itemsets_fpgrowth, 0.001)
print(data_first_rule_fpgrowth.head())

            antecedents           consequents  antecedent support  \
0  (30 seconds to mars)        (fall out boy)            0.029598   
1        (fall out boy)  (30 seconds to mars)            0.038760   
2  (30 seconds to mars)         (linkin park)            0.029598   
3         (linkin park)  (30 seconds to mars)            0.083862   
4                (muse)  (30 seconds to mars)            0.174066   

   consequent support   support  confidence      lift  leverage  conviction  
0            0.038760  0.010571    0.357143  9.214286  0.009424    1.495263  
1            0.029598  0.010571    0.272727  9.214286  0.009424    1.334302  
2            0.083862  0.014799    0.500000  5.962185  0.012317    1.832276  
3            0.029598  0.014799    0.176471  5.962185  0.012317    1.178345  
4            0.029598  0.012685    0.072874  2.462117  0.007533    1.046678  
   antecedents  consequents  antecedent support  consequent support   support  \
0       (muse)  (radiohead)         

In [17]:
def first_rule(data, sup, lift):
    response = data[
        (data['antecedent support'] >= sup) &
        (data['lift'] >= lift)
    ]
    return response.loc[:,['antecedents','consequents']]

"""
    Для популярной группы/исполнителя подберем подобрать похожие группы/артистов
"""
print_res("Apriori", first_rule(data_first_rule_apriori,0.2,3))
print_res("FPGrowth", first_rule(data_first_rule_fpgrowth,0.05,1.5))

Algorithm: Apriori
{'radiohead'}  ->  {'thom yorke'}
{'radiohead'}  ->  {'beck', 'air'}
{'radiohead'}  ->  {'nirvana', 'air'}
{'radiohead'}  ->  {'coldplay', 'antony and the johnsons'}
{'radiohead'}  ->  {'david bowie', 'arcade fire'}
{'radiohead'}  ->  {'death cab for cutie', 'arctic monkeys'}
{'radiohead'}  ->  {'beck', 'portishead'}
{'radiohead'}  ->  {'muse', 'björk'}
{'radiohead'}  ->  {'placebo', 'björk'}
{'radiohead'}  ->  {'sigur rós', 'björk'}
{'radiohead'}  ->  {'coldplay', 'blur'}
{'radiohead'}  ->  {'blur', 'franz ferdinand'}
{'radiohead'}  ->  {'coldplay', 'eels'}
{'radiohead'}  ->  {'coldplay', 'joy division'}
{'radiohead'}  ->  {'coldplay', 'pj harvey'}
{'radiohead'}  ->  {'coldplay', 'portishead'}
{'radiohead'}  ->  {'coldplay', 'sufjan stevens'}
{'radiohead'}  ->  {'coldplay', 'the doors'}
{'radiohead'}  ->  {'pink floyd', 'daft punk'}
{'radiohead'}  ->  {'mogwai', 'explosions in the sky'}
{'radiohead'}  ->  {'franz ferdinand', 'interpol'}
{'radiohead'}  ->  {'queens o

#### 2 правило:

In [18]:
def data_for_second_rule(data_for_this_rule, min_t):
    new_data = association_rules(data_for_this_rule, metric="confidence", min_threshold=min_t) 
    new_data["antecedent_len"] = new_data["antecedents"].apply(lambda x: len(x))
    new_data["consequent_len"] = new_data["consequents"].apply(lambda x: len(x))
    return new_data 

frequent_itemsets_fpgrowth = fpgrowth(data_to_process, min_support=0.008, use_colnames=True)

data_second_rule_apriori = data_for_second_rule(frequent_itemsets_apriori,0.5)
print(data_second_rule_apriori.head())

data_second_rule_fpgrowth = data_for_second_rule(frequent_itemsets_fpgrowth, 0.005)
print(data_second_rule_fpgrowth.head())

            antecedents    consequents  antecedent support  \
0  (30 seconds to mars)  (linkin park)            0.029598   
1      (65daysofstatic)       (mogwai)            0.019732   
2                (abba)      (madonna)            0.025370   
3                 (air)    (radiohead)            0.092319   
4       (amy macdonald)     (coldplay)            0.022551   

   consequent support   support  confidence       lift  leverage  conviction  \
0            0.083862  0.014799    0.500000   5.962185  0.012317    1.832276   
1            0.051445  0.010571    0.535714  10.413405  0.009556    2.043042   
2            0.063425  0.012685    0.500000   7.883333  0.011076    1.873150   
3            0.237491  0.048626    0.526718   2.217840  0.026701    1.611107   
4            0.188161  0.011980    0.531250   2.823385  0.007737    1.731924   

   antecedent_len  consequent_len  
0               1               1  
1               1               1  
2               1               1  
3 

In [19]:
def second_rule(data, antecedent_len):
    get_mean_confid = data['confidence'].mean()
    response = data[
        (data['antecedent_len'] > antecedent_len) &
        (data['consequent_len'] == 1) &
        (data['confidence'] > (1 - (get_mean_confid)/2))
    ]
    return response.loc[:,['antecedents','consequents']]

"""
 По нескольким  группам определить наиболее вероятную группу, которая может понравится пользователю
"""    
print_res("Apriori", second_rule(data_second_rule_apriori,3))
print_res("FPGrowth", second_rule(data_second_rule_fpgrowth,4))

Algorithm: Apriori
{'arctic monkeys', 'muse', 'coldplay', 'oasis'}  ->  {'bloc party'}
{'arctic monkeys', 'muse', 'bloc party', 'oasis'}  ->  {'coldplay'}
{'arctic monkeys', 'bloc party', 'coldplay', 'oasis'}  ->  {'muse'}
{'coldplay', 'muse', 'bloc party', 'oasis'}  ->  {'arctic monkeys'}
{'arctic monkeys', 'muse', 'the killers', 'coldplay'}  ->  {'bloc party'}
{'arctic monkeys', 'muse', 'coldplay', 'bloc party'}  ->  {'the killers'}
{'arctic monkeys', 'muse', 'the killers', 'bloc party'}  ->  {'coldplay'}
{'arctic monkeys', 'the killers', 'coldplay', 'bloc party'}  ->  {'muse'}
{'arctic monkeys', 'muse', 'the strokes', 'coldplay'}  ->  {'bloc party'}
{'arctic monkeys', 'muse', 'the strokes', 'bloc party'}  ->  {'coldplay'}
{'arctic monkeys', 'the strokes', 'coldplay', 'bloc party'}  ->  {'muse'}
{'coldplay', 'muse', 'the strokes', 'bloc party'}  ->  {'arctic monkeys'}
{'arctic monkeys', 'the killers', 'coldplay', 'oasis'}  ->  {'bloc party'}
{'arctic monkeys', 'bloc party', 'coldplay

#### 3 правило:

In [20]:
def third_rule(data, group):
    get_mean_confid =  response = data[(data['antecedents'] == group)]['confidence'].mean()
    response = data[
        (data['antecedents'] == group) &
        (data['confidence'] >= get_mean_confid)
    ]
    return response.loc[:,['antecedents','consequents']]
    
"""
 По нескольким  группам определить наиболее вероятную группу, которая может понравится подьзователю
"""
group_or_artist = {'30 seconds to mars'}
print_res("Apriori", third_rule(data_second_rule_apriori,group_or_artist))
print_res("FPGrowth", third_rule(data_second_rule_fpgrowth,group_or_artist))

Algorithm: Apriori
{'30 seconds to mars'}  ->  {'linkin park'}
Algorithm: FPGrowth
{'30 seconds to mars'}  ->  {'muse'}
{'30 seconds to mars'}  ->  {'linkin park'}


#### 4 правило:

In [21]:
frequent_itemsets_fpmax = fpmax(data_to_process, min_support=0.05,max_len=3, use_colnames=True)
frequent_itemsets_fpmax

,support,itemsets
0,0.050035,(gorillaz)
1,0.050035,(pj harvey)
2,0.050035,(editors)
3,0.050740,(kaiser chiefs)
4,0.050740,(snow patrol)
...,...,...
67,0.054968,"(pink floyd, radiohead)"
68,0.070472,"(the beatles, radiohead)"
69,0.068358,"(coldplay, muse)"
70,0.080338,"(muse, radiohead)"


In [22]:
def forth_rule(data_for_this_rule, min_t):
    return association_rules(data_for_this_rule, metric="support",support_only=True, min_threshold=min_t) 

data_forth_rule_fpmax = forth_rule(frequent_itemsets_fpmax,0.06)
data_forth_rule_fpmax.loc[:,['antecedents','consequents','support']]

,antecedents,consequents,support
0,(the beatles),(radiohead),0.070472
1,(radiohead),(the beatles),0.070472
2,(coldplay),(muse),0.068358
3,(muse),(coldplay),0.068358
4,(muse),(radiohead),0.080338
5,(radiohead),(muse),0.080338
6,(coldplay),(radiohead),0.083157
7,(radiohead),(coldplay),0.083157


In [23]:
"""
Определение наиболее часто всречающихся групп/артистов и их похожих групп/артистов
"""
print_res("FPmax", data_forth_rule_fpmax)

Algorithm: FPmax
{'the beatles'}  ->  {'radiohead'}
{'radiohead'}  ->  {'the beatles'}
{'coldplay'}  ->  {'muse'}
{'muse'}  ->  {'coldplay'}
{'muse'}  ->  {'radiohead'}
{'radiohead'}  ->  {'muse'}
{'coldplay'}  ->  {'radiohead'}
{'radiohead'}  ->  {'coldplay'}


#### 5 правило:

In [33]:
def data_for_fifth_rule(data_for_this_rule):
    new_data = association_rules(data_for_this_rule, metric="conviction", min_threshold=0.8) 
    new_data["antecedent_len"] = new_data["antecedents"].apply(lambda x: len(x))
    new_data["consequent_len"] = new_data["consequents"].apply(lambda x: len(x))
    return new_data 

frequent_itemsets_fpgrowth = fpgrowth(data_to_process_with_sex, min_support=0.01, use_colnames=True)
frequent_itemsets_apriori = apriori(data_to_process_with_sex, min_support=0.01, use_colnames=True)

data_fifth_rule_apriori = data_for_fifth_rule(frequent_itemsets_apriori)
print(data_fifth_rule_apriori.head())

data_fifth_rule_fpgrowth = data_for_fifth_rule(frequent_itemsets_fpgrowth)
print(data_fifth_rule_fpgrowth.head())

            antecedents           consequents  antecedent support  \
0                   (m)        (3 doors down)            0.780127   
1  (30 seconds to mars)        (fall out boy)            0.029598   
2        (fall out boy)  (30 seconds to mars)            0.038760   
3  (30 seconds to mars)         (linkin park)            0.029598   
4         (linkin park)  (30 seconds to mars)            0.083862   

   consequent support   support  confidence      lift  leverage  conviction  \
0            0.020437  0.013390    0.017164  0.839828 -0.002554    0.996669   
1            0.038760  0.010571    0.357143  9.214286  0.009424    1.495263   
2            0.029598  0.010571    0.272727  9.214286  0.009424    1.334302   
3            0.083862  0.014799    0.500000  5.962185  0.012317    1.832276   
4            0.029598  0.014799    0.176471  5.962185  0.012317    1.178345   

   antecedent_len  consequent_len  
0               1               1  
1               1               1  
2 

In [46]:
def fifth_rule(data,min_sup,sex):
    response = data[ (data['antecedents']== sex) 
                    & (data['antecedent_len'] == 1) 
                    & (data['consequent_len'] == 1) 
                    & (data['consequent support'] > min_sup)
    ]
    get_max_confid = response['confidence'].max()
    response = response[response['confidence'] == get_max_confid]
    return response.loc[:,['antecedents','consequents']]

"""
 Определить самую популярную группу у определенного пола
"""    
male = frozenset('m')
female = frozenset('f')
min_support = 0.09
print_res("Apriori", fifth_rule(data_fifth_rule_apriori,min_support , male))
print_res("FPGrowth", fifth_rule(data_fifth_rule_fpgrowth,min_support , male))

Algorithm: Apriori
{'m'}  ->  {'radiohead'}
Algorithm: FPGrowth
{'m'}  ->  {'radiohead'}


In [47]:
print_res("Apriori", fifth_rule(data_fifth_rule_apriori,min_support , female))
print_res("FPGrowth", fifth_rule(data_fifth_rule_fpgrowth,min_support , female))

Algorithm: Apriori
{'f'}  ->  {'coldplay'}
{'f'}  ->  {'muse'}
Algorithm: FPGrowth
{'f'}  ->  {'muse'}
{'f'}  ->  {'coldplay'}
